In [3]:
!pip install -q lxml
!pip install webdriver-manager

import bs4
import lxml
import pandas
import urllib

from urllib import request



In [4]:
url_ligue_1 = "https://fr.wikipedia.org/wiki/Championnat_de_France_de_football_2019-2020"
    
request_text = request.urlopen(url_ligue_1).read()
#print(request_text[:1000])
page = bs4.BeautifulSoup(request_text, "lxml")


In [5]:
print(page.find("title"))

<title>Championnat de France de football 2019-2020 — Wikipédia</title>


In [6]:
#trouver le tableau
# on identifie le tableau en question : c'est le premier qui a cette classe "wikitable sortable"
tableau_participants = page.find('table', {'class' : 'wikitable sortable'})
#print(tableau_participants)

In [ ]:
#On recherche d’abord toutes les lignes du tableau avec la balise tr
table_body = tableau_participants.find('tbody')
rows = table_body.find_all('tr')

In [ ]:
#On obtient une liste où chaque élément est une des lignes du tableau Pour illustrer cela, on va d’abord afficher la première ligne. 
#print(rows[0])

In [ ]:
#La seconde ligne
#print(rows[1])

In [ ]:
# EXEMPLE Nettoyer les sorties en ne gardant que le texte sur une ligne
cols = rows[1].find_all('td')
#print(cols[0])
#print(cols[0].text.strip())
for ele in cols : 
    #print(ele.text.strip())

In [ ]:
# GENERALISATION
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    print(cols)

In [ ]:
#On va mettre tout le contenu dans un dictionnaire, pour le transformer ensuite en DataFrame pandas :
dico_participants = dict()
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    if len(cols) > 0 : 
        dico_participants[cols[0]] = cols[1:]
dico_participants

In [ ]:
# Formation nouveau tableau
data_participants = pandas.DataFrame.from_dict(dico_participants,orient='index')
data_participants.head()

In [ ]:
# récupération entête